## This notebook is belong to my research bakeries of Minsk

In [2]:
# !pip install geopy
# !pip install folium
# !pip install transliterate

In [3]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
import json
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import plotly.express as px
import plotly.graph_objects as go
from transliterate import translit

### Introduction

    When I was looking about topics of researches I really want to use my native town at this task and I chose the topic "Which Minsk district really need more bakeries?". 
    This topic I decided to research also because I really like morning's croissant with fresh coffee and the same time I'm planning to move and I'll know which district have enough it now.  

##### Target audience of this research:
    It's definitely people who interested in opening bakery and looking for district to open place. In my mind this person should consider how many bakeries Minsk has now and how peoples rated it. Opening hours in my opinion also important because I and my colleagues and friends really likes to buy fresh croissant and morning coffee when we're going to work. 

##### Plan of my research:
    1. Download the data – this part is concluding creating of data frame with information about districts of Minsk and it coordinates. 
    2. Using API Foursquare I should to know id of bakeries category for searching all company  this type and full their information. *
    3. Join all data into one data frame including information about each company and it’s district. 
    4. Deleting duplicate rows, transforming columns.
    5. Analyze it and appending on map. 
    6. Using K-Means create model and fit it on data frame. 
    7. Give a conclusion.  
    
 
    * Because the free part of API service is limited by calls.

### Data 

##### Data description of research:
    Open source data about Minsk districts from yandex map.
    API foursquare to find all bakeries of Minsk.    

### Methodology section

    Here we become to adding the main data about our research including list of bakeries and information about every place which we'll be analysing. 
    I will use pandas library and it's methods to analysing the data. And also I'll use K-means technique to separate my data to different clusters.

    Using libraries: pandas, numpy, requests, json, geodesic, translit etc.
    Machine learning techniques: K-means for clustering data.

##### Add Minsk and its districts

In [4]:
address = 'Minsk, Belarus'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Minsk are {}, {}.'.format(latitude, longitude))

minsk_districts = [
     ['Центральный район', 53.935766, 27.521649],
     ['Советский район', 53.940933, 27.585951], 
     ['Первомайский район', 53.940165, 27.646695],
     ['Партизанский район', 53.902873, 27.634289],
     ['Заводской район', 53.869370, 27.647405],
     ['Ленинский район', 53.859626, 27.585654],
     ['Фрунзенский район', 53.906236, 27.453907],
     ['Октябрьский район', 53.858930, 27.538223],
     ['Московский район', 53.871021, 27.492966],
]
minsk_districts_df = pd.DataFrame(minsk_districts, columns = ['district','dist_lat','dist_lng' ])
minsk_districts_df

The geograpical coordinate of Minsk are 53.902334, 27.5618791.


,district,dist_lat,dist_lng
0,Центральный район,53.935766,27.521649
1,Советский район,53.940933,27.585951
2,Первомайский район,53.940165,27.646695
3,Партизанский район,53.902873,27.634289
4,Заводской район,53.869370,27.647405
5,Ленинский район,53.859626,27.585654
6,Фрунзенский район,53.906236,27.453907
7,Октябрьский район,53.858930,27.538223
8,Московский район,53.871021,27.492966


##### Create a map of Minsk with districts superimposed on top

In [37]:
# create map of Minsk using latitude and longitude values
map_minsk = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, name in zip(minsk_districts_df['dist_lat'], minsk_districts_df['dist_lng'], minsk_districts_df['district']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF1818',
        fill_opacity=0.9,
        parse_html=False).add_to(map_minsk)  
    
map_minsk

##### Connecting API  Foursquare

In [18]:
CLIENT_ID = '*****' 
CLIENT_SECRET = '********' 
VERSION = '20180605' 

Your credentails:
CLIENT_ID: ZHOP3QTSLXALEWARFJW44311ULGDBQCZHSASBS52XKCPLXHW
CLIENT_SECRET:D1XBVOX34ISMLWKH5UDKFQIJPCHBKYHQOFORO50K0G5ZHOMX


##### Getting category for our research

In [7]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)

In [8]:
categories = requests.get(url).json()
food_category = pd.DataFrame(categories['response']['categories'][3]['categories'])
id_bakery = food_category.loc[food_category['name'] == 'Bakery'].values[0][0]

,id,name,pluralName,shortName,icon,categories
0,503288ae91d4c4b30a586d67,Afghan Restaurant,Afghan Restaurants,Afghan,{'prefix': 'https://ss3.4sqi.net/img/categorie...,[]
1,4bf58dd8d48988d1c8941735,African Restaurant,African Restaurants,African,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '4bf58dd8d48988d10a941735', 'name': 'E..."
2,4bf58dd8d48988d14e941735,American Restaurant,American Restaurants,American,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N..."
3,4bf58dd8d48988d142941735,Asian Restaurant,Asian Restaurants,Asian,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '56aa371be4b08b9a8d573568', 'name': 'B..."
4,4bf58dd8d48988d169941735,Australian Restaurant,Australian Restaurants,Australian,{'prefix': 'https://ss3.4sqi.net/img/categorie...,[]


##### Getting list of Minsk's bakeries

In [10]:
def create_url(lat,lng):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    id_bakery, 
    100)
    return url

In [11]:
baker_list = []
for index, row in minsk_districts_df.iterrows():
    url = create_url(row['dist_lat'],row['dist_lng'])
    district_bakeries = requests.get(url).json()

    for item in district_bakeries['response']['groups'][0]['items']:
        baker_list.append({
        'district': row['district'],
        'summary': item['reasons']['items'][0]['summary'],
        'id': item['venue']['id'],
        'name': item['venue']['name'],
        'lat': item['venue']['location']['lat'],
        'lng': item['venue']['location']['lng'],
        'categories': item['venue']['categories'][0]['name']
        })
        
baker_df = pd.DataFrame(baker_list)
baker_df = baker_df.join(minsk_districts_df.set_index('district'), on='district')
baker_df.head()

,district,summary,id,name,lat,lng,categories,dist_lat,dist_lng
0,Центральный район,This spot is popular,5473097b498e2d4d05afda1b,Пан Круассан,53.929749,27.541323,Bakery,53.935766,27.521649
1,Центральный район,This spot is popular,59ee1ae8d0a1496688e63150,Brioche Bistro,53.939274,27.478426,Bakery,53.935766,27.521649
2,Центральный район,This spot is popular,58ba97964bc2f1632a353e04,Stories,53.900877,27.552514,Bakery,53.935766,27.521649
3,Центральный район,This spot is popular,522dc5b311d2920f2460d44c,Cakes.by,53.900149,27.522044,Bakery,53.935766,27.521649
4,Центральный район,This spot is popular,4f7afb6ae4b02579039ddc3f,Зерно,53.912388,27.580818,Bakery,53.935766,27.521649


##### Exclude from our df duplicated rows through by district and id of bakery

In [12]:
distance = []
for index, row in baker_df.iterrows():
    dist = geodesic([row['lat'],row['lng']], [row['dist_lat'],row['dist_lng']], ellipsoid='WGS-84').m
    distance.append(dist)
    
baker_df['distance'] = distance


In [14]:
baker_df.sort_values(["id",'distance'], inplace = True)
baker_df.drop_duplicates(subset ="id", keep = 'first', inplace = True)
baker_df.reset_index(drop=True, inplace=True)

##### Append to our df latin names of our bakeries because folium doesn't work correct with cyrillic words

In [13]:
latin_name = []
for index, row in baker_df.iterrows():
    latin = translit(row['name'], "ru", reversed=True)
    latin_name.append(latin)
    
baker_df['latin_name'] = latin_name

##### Add to the map our bakeries

In [38]:
map_minsk = folium.Map(location=[latitude, longitude], zoom_start=12) 


# add markers to map
for lat, lng, name in zip(minsk_districts_df['dist_lat'], minsk_districts_df['dist_lng'], minsk_districts_df['district']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF1818',
        fill_opacity=0.9,
        parse_html=False).add_to(map_minsk)  


# add markers to map
for lat, lng, name in zip(baker_df['lat'], baker_df['lng'], baker_df['latin_name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_minsk)  
    
map_minsk

##### Add info about each bakery like 'rating', 'price','likes' etc.

In [42]:
#bakery_full = []
for bakery_id in baker_df['id'].unique():
    if bakery_id not in bakery_full_df['id'].unique():
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        bakery_id,   
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION)
        full_inf_bakeries = requests.get(url).json()   
        item = full_inf_bakeries['response']['venue']
        bakery_full.append({
        'id': bakery_id,
        'likes': item['likes']['count'],
        'dislike': item['dislike'],
        'rating': item['rating'] if 'rating' in item else '',        
        'url': item['url'] if 'url' in item else '',
        'price': item['price']['tier'] if 'price' in item else '',
        'ok': item['ok'],
        'ratingSignals': item['ratingSignals'] if 'ratingSignals' in item else '',
        'specials': item['specials']['count'],
        'tips': item['tips']['count'],
        'listed': item['listed']['count']
            }) 

In [51]:
bakery_full_df = pd.DataFrame(bakery_full)
bakery_full_df = bakery_full_df.join(baker_df.set_index('id'), on='id')
bakery_full_df.replace("", 0, inplace = True)
print(bakery_full_df.shape)

(88, 21)


##### Looking at the part of data frame with some columns to analyze

In [59]:
bakery_full_df[['name','district','categories','rating','price','tips']]

,name,district,categories,rating,price,tips
12,Горячий хлеб,Центральный район,Bakery,0.0,0,1
13,Блинная,Первомайский район,Bakery,0.0,0,1
15,Магазин-кафетерий,Центральный район,Dessert Shop,0.0,0,1
16,Дольче,Московский район,Bakery,0.0,0,1
17,Свежий хлеб,Октябрьский район,Bakery,0.0,0,2
...,...,...,...,...,...,...
32,Пан Круассан,Центральный район,Bakery,8.4,0,13
84,Территория Кофе,Советский район,Coffee Shop,8.4,0,5
55,Stories,Центральный район,Bakery,8.5,0,81
57,Brø Bakery,Партизанский район,Bakery,8.7,0,26


##### And now we're prepared our data to analyze

Let's first look how many bakeries we have from each district and how many types of bakeries we have from each categories.

In [52]:
categories_labels = bakery_full_df['categories'].value_counts().index
categories_values = bakery_full_df['categories'].value_counts().values

fig = px.pie(bakery_full_df, values=categories_values, names=categories_labels,
            color_discrete_sequence=px.colors.sequential.Bluyl)
fig.update_layout(
    title = 'Count of places by categories')
fig.show()

In [53]:
district_labels = bakery_full_df['district'].value_counts().index
district_values = bakery_full_df['district'].value_counts().values

fig = px.pie(bakery_full_df, values=district_values, names=district_labels,
            color_discrete_sequence=px.colors.sequential.Bluyl)
fig.update_layout(
    title = 'Count of places by district')
fig.show()

##### Look at the out frame grouping by district and lookin mean values

In [62]:
bakery_full_df.groupby('district').mean().sort_values(["rating",'price'])

,likes,dislike,rating,price,ok,ratingSignals,specials,tips,listed,lat,lng,dist_lat,dist_lng,distance
district,,,,,,,,,,,,,,
Фрунзенский район,7.000000,False,3.525000,0.000000,False,11.500000,0.0,2.000000,1.500000,53.902638,27.459143,53.906236,27.453907,1959.875968
Октябрьский район,64.250000,False,3.612500,0.250000,False,96.625000,0.0,22.000000,14.000000,53.883407,27.543747,53.858930,27.538223,2869.849479
Центральный район,77.576923,False,4.092308,0.192308,False,133.461538,0.0,36.461538,16.384615,53.910286,27.542295,53.935766,27.521649,3554.805297
Заводской район,8.000000,False,4.325000,0.000000,False,14.250000,0.0,3.500000,0.000000,53.866953,27.655436,53.869370,27.647405,1423.422825
Советский район,59.347826,False,4.404348,0.173913,False,94.391304,0.0,24.130435,12.347826,53.924046,27.584748,53.940933,27.585951,2129.749178
Первомайский район,10.222222,False,4.433333,0.000000,False,18.222222,0.0,4.444444,0.666667,53.944241,27.653939,53.940165,27.646695,1718.138262
Московский район,80.200000,False,5.940000,0.400000,False,117.000000,0.0,33.200000,17.600000,53.875364,27.499770,53.871021,27.492966,2769.710420
Партизанский район,85.222222,False,5.955556,0.222222,False,147.666667,0.0,34.222222,16.333333,53.909168,27.595053,53.902873,27.634289,3315.887513


Now let's look mean rating through by every district

In [64]:
import plotly.graph_objects as go
labels = bakery_full_df['district'].value_counts().index
values_rat = bakery_full_df.groupby('district').mean()['rating']
values_likes = bakery_full_df.groupby('district').mean()['likes']
count_places = bakery_full_df['district'].value_counts().values

fig = go.Figure(data=[
    go.Bar(name='Mean rating', x=labels, y=values_rat,text=list(map('{:.2f}'.format,values_rat)),
           textposition='auto',
           marker_color='SeaGreen'),
    go.Bar(name='Count places', x=labels, y=count_places,
           text=list(map('{:.2f}'.format,count_places)), textposition='auto',
           marker_color='Khaki')
])
# Change the bar mode
fig.update_layout(barmode='group', title = 'Mean rating and count likes of places by district')
fig.show()

##### As we've seen the plot higher:
    1. 'Центральный район' has most of bakeries in Minsk.
    2. 'Партизанский район' has highest rating bakeries in Minsk.


##### And now let's clustering our bakeries using K-means algorithm.

In [65]:
kclusters = 5
bakery_full_df_clustering = bakery_full_df[['likes','rating','price','ratingSignals','specials','tips','listed']]
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bakery_full_df_clustering)

##### And create new df join the fitted clusters and id of bakeries.

In [66]:
bakery_full_df['cluster'] = kmeans.labels_

The count of bakeries in clusters you can show below:

In [95]:
bakery_full_df.groupby('cluster').count()['id']

cluster
0    73
1     1
2     3
3     2
4     9
Name: id, dtype: int64

In [92]:
bakery_full_df.groupby('cluster').mean()

,likes,dislike,rating,price,ok,ratingSignals,specials,tips,listed,lat,lng,dist_lat,dist_lng,distance
cluster,,,,,,,,,,,,,,
0,12.054795,False,3.946575,0.000000,False,19.342466,0.0,5.205479,3.410959,53.912398,27.571215,53.919370,27.566475,2505.285722
1,1011.000000,False,7.900000,2.000000,False,1541.000000,0.0,357.000000,229.000000,53.912388,27.580818,53.940933,27.585951,3195.005240
2,355.000000,False,7.666667,2.000000,False,551.333333,0.0,125.333333,70.000000,53.878120,27.537466,53.877608,27.555159,2751.988208
3,436.000000,False,6.300000,1.000000,False,809.500000,0.0,235.000000,29.000000,53.903553,27.555421,53.935766,27.521649,4241.786574
4,154.222222,False,6.622222,0.555556,False,263.222222,0.0,70.222222,35.888889,53.907192,27.566023,53.925376,27.566340,3803.949240


As we've seen our bakeries is separated by 5 different clusters and the best places in 1 and 2 cluster, because it has the best rating and likes.
I definately going to visit this places because it has the best ratings and likes. 

In [69]:
bakery_full_df[bakery_full_df['cluster'] == 1]

,id,likes,dislike,rating,url,price,ok,ratingSignals,specials,tips,...,summary,name,lat,lng,categories,dist_lat,dist_lng,distance,latin_name,cluster
8,4f7afb6ae4b02579039ddc3f,1011,False,7.9,http://cafezerno.by,2,False,1541,0,357,...,This spot is popular,Зерно,53.912388,27.580818,Bakery,53.940933,27.585951,3195.00524,Zerno,1


In [70]:
bakery_full_df[bakery_full_df['cluster'] == 2]

,id,likes,dislike,rating,url,price,ok,ratingSignals,specials,tips,...,summary,name,lat,lng,categories,dist_lat,dist_lng,distance,latin_name,cluster
33,54fee005498e8e467263436b,340,False,7.0,http://www.cafezerno.by,2,False,588,0,103,...,This spot is popular,Зерно,53.908389,27.581433,Coffee Shop,53.902873,27.634289,3527.768989,Zerno,2
0,4bfcf7b58992a593595fabb0,399,False,7.3,http://cafe-bakery.relax.by,2,False,601,0,142,...,This spot is popular,Бейкери дю Солей,53.876255,27.553004,Bakery,53.858930,27.538223,2159.566939,Bejkeri dju Solej,2
19,50f17208e4b08ef910e45079,326,False,8.7,http://moulin.relax.by,2,False,465,0,131,...,This spot is popular,Moulin,53.849716,27.477960,Bakery,53.871021,27.492966,2568.628695,Moulin,2


In [68]:
bakery_full_df[bakery_full_df['cluster'] == 0]

,id,likes,dislike,rating,url,price,ok,ratingSignals,specials,tips,...,summary,name,lat,lng,categories,dist_lat,dist_lng,distance,latin_name,cluster
12,500209b9e4b0382b9ff4bff1,3,False,0.0,http://www.minskhleb.by/shop/magaziny/kropotki...,0,False,0,0,1,...,This spot is popular,Горячий хлеб,53.915330,27.556110,Bakery,53.935766,27.521649,3209.216204,Gorjachij hleb,0
13,50053087e4b067e568e868a4,1,False,0.0,0,0,False,0,0,1,...,This spot is popular,Блинная,53.947467,27.697553,Bakery,53.940165,27.646695,3437.002784,Blinnaja,0
15,5034a5efe4b0df9e2d41e90f,1,False,0.0,0,0,False,0,0,1,...,This spot is popular,Магазин-кафетерий,53.905548,27.546128,Dessert Shop,53.935766,27.521649,3728.122070,Magazin-kafeterij,0
16,5052fc61e4b0e932c7c1e6e1,0,False,0.0,0,0,False,0,0,1,...,This spot is popular,Дольче,53.897342,27.531783,Bakery,53.871021,27.492966,3885.600282,Dol'che,0
17,5061818c498e317f7b87b976,2,False,0.0,0,0,False,0,0,2,...,This spot is popular,Свежий хлеб,53.863331,27.524038,Bakery,53.858930,27.538223,1054.003230,Svezhij hleb,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,566d54ce498e2d31443a01c7,11,False,8.1,https://vk.com/hata_cake_coffee,0,False,17,0,5,...,This spot is popular,hata,53.912347,27.678615,Bakery,53.902873,27.634289,3098.129277,hata,0
49,57ca89d5498e98eee9c81eea,35,False,8.1,http://dolcelux.by,0,False,52,0,11,...,This spot is popular,Dolce,53.849513,27.478079,Bakery,53.871021,27.492966,2586.506701,Dolce,0
32,5473097b498e2d4d05afda1b,47,False,8.4,http://pankruassan.relax.by,0,False,62,0,13,...,This spot is popular,Пан Круассан,53.929749,27.541323,Bakery,53.935766,27.521649,1455.442338,Pan Kruassan,0
84,5daaf67ff74cbf0008dbe6ac,6,False,8.4,0,0,False,10,0,5,...,This spot is popular,Территория Кофе,53.927440,27.606528,Coffee Shop,53.940933,27.585951,2020.387943,Territorija Kofe,0


In [71]:
bakery_full_df[bakery_full_df['cluster'] == 3]

,id,likes,dislike,rating,url,price,ok,ratingSignals,specials,tips,...,summary,name,lat,lng,categories,dist_lat,dist_lng,distance,latin_name,cluster
25,530665c1498e8117a1d48222,406,False,5.8,http://foodrepublic.by,0,False,853,0,274,...,This spot is popular,Food Republic,53.905460,27.561701,Food Court,53.935766,27.521649,4278.164281,Food Republic,3
28,5374c450498e4b7f19d54b03,466,False,6.8,0,2,False,766,0,196,...,This spot is popular,Cinnabon,53.901646,27.549141,Bakery,53.935766,27.521649,4205.408868,Cinnabon,3


In [72]:
bakery_full_df[bakery_full_df['cluster'] == 4]

,id,likes,dislike,rating,url,price,ok,ratingSignals,specials,tips,...,summary,name,lat,lng,categories,dist_lat,dist_lng,distance,latin_name,cluster
18,50a7679fe4b041c065193a27,94,False,5.7,http://stolle.by,0,False,169,0,31,...,This spot is popular,Штолле / Stolle,53.903771,27.558714,Bakery,53.935766,27.521649,4314.167636,Shtolle / Stolle,4
1,4ccb0b9c54f0b1f77ce60cca,115,False,5.8,https://www.minskhleb.by,0,False,225,0,74,...,This spot is popular,Каравай,53.908001,27.575787,Bakery,53.902873,27.634289,3887.107119,Karavaj,4
4,4df8955818385456c7f514e5,181,False,6.0,http://stolle.by,2,False,352,0,112,...,This spot is popular,Штолле / Stolle,53.917542,27.586509,Bakery,53.940933,27.585951,2603.799832,Shtolle / Stolle,4
56,58c2d2476ad5a14069506f76,136,False,6.2,https://uhg.by/restaurants/kafe-pekarnya-paul/,0,False,253,0,59,...,This spot is popular,PAUL,53.903717,27.558369,Bakery,53.935766,27.521649,4306.410946,PAUL,4
7,4f05bdf4e3002adb768a5731,86,False,6.3,http://stolle.by/%20,0,False,158,0,45,...,This spot is popular,Штолле / Stolle,53.907789,27.583579,Bakery,53.902873,27.634289,3377.477964,Shtolle / Stolle,4
24,528f768e498e55fd2ae00727,134,False,6.5,http://stolle.by,0,False,217,0,45,...,This spot is popular,Штолле / Stolle,53.907758,27.575317,Bakery,53.902873,27.634289,3913.855396,Shtolle / Stolle,4
5,4e1056e6c65b4a49f303033d,226,False,6.6,http://stolle.by,0,False,380,0,120,...,This spot is popular,Штолле / Stolle,53.905094,27.546775,Bakery,53.935766,27.521649,3792.066272,Shtolle / Stolle,4
35,559d1ed5498e914e9c52a64d,213,False,8.0,0,3,False,315,0,65,...,This spot is popular,BRIOCHE Paris,53.910180,27.556638,Dessert Shop,53.935766,27.521649,3659.748357,BRIOCHE Paris,4
55,58ba97964bc2f1632a353e04,203,False,8.5,0,0,False,300,0,81,...,This spot is popular,Stories,53.900877,27.552514,Bakery,53.935766,27.521649,4380.909637,Stories,4


In [73]:
map_minsk = folium.Map(location=[latitude, longitude], zoom_start=12) 

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lng, name, cluster in zip(bakery_full_df['lat'],
                                   bakery_full_df['lng'],
                                   bakery_full_df['latin_name'],
                                   bakery_full_df['cluster']):
    label = folium.Popup(str(name) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_minsk)  
    
map_minsk

### Result and discussion section

    Answering to the question 'Which Minsk district really need more bakeries?' I should notice that most count of bakeries belong to the center of city and central district. And it's okay because most of cultural and evening life are using there, all European cities has the same statistic. But according to my question - bakery for me, it's firstly the morning coffee and croissant. And I defiantly recommend to open bakeries not in the center of city, but in the new districts far from the center. Really great example we can see at 'Moulin' bakeries which is according to the high 2 cluster. From all 87 Minsk's bakeries just 3 from it has 2 cluster and it place is situated to new district.